# 接口：从协议到抽象类

从鸭子类型的代表特征动态协议，到使接口更 明确、能验证实现是否符合规定的抽象基类(Abstract Base Class， ABC)。

## 鸭子类型特征

什么是鸭子类型？

即忽略对象的真正类型，转而关注对象有没有实现所需的方法、签名和语义

本章讨论的主题是“鸭子类型”:对象的类型无关紧要，只要实现 了特定的协议即可。

## 11.3 打猴子补丁


In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]



In [ ]:
deck = FrenchDeck()
deck[:5]

In [ ]:
# 没有实现序列赋值，因此无法使用随机方法就地打乱
from random import shuffle

shuffle(deck)

In [ ]:
# 猴子补丁
# 定义一个普通的静态函数：

def setCard(deck, position, value):
    deck._cards[position] = value

# 动态打猴子补丁
FrenchDeck.__setitem__ = setCard
shuffle(deck)
deck[:5]

这里的关键是，set_card 函数要知道 deck 对象有一个名为 _cards 的属性，而且 _cards 的值必须是可变序列。然后，我们把 set_card
函数赋值给特殊方法 __setitem__，从而把它依附到 FrenchDeck
类上。这种技术叫猴子补丁:在运行时修改类或模块，而不改动源码。 猴子补丁很强大，但是打补丁的代码与要打补丁的程序耦合十分紧密， 而且往往要处理隐藏和没有文档的部分。
除了举例说明猴子补丁之外，示例 11-6 还强调了协议是动态 的:random.shuffle 函数不关心参数的类型，只要那个对象实现了
部分可变序列协议即可。即便对象一开始没有所需的方法也没关系，后
来再提供也行。

## 抽象基类

抽象基类的本质就是几个特殊方法

In [ ]:
class Struggle:
    def __len__(self):
        return 1

        
# 抽象基类的本质就是几个特殊方法
from collections import Sized
isinstance(Struggle(), Sized)

然而，即便是抽象基类，也不能滥用 isinstance 检查，用得多了可能导致代码异味，即表明面向对象设计得不好。在一连串 if/elif/elif 中使用 isinstance 做检查，然后根据对象的类型执
行不同的操作，通常是不好的做法;此时应该使用多态，即采用一定的 方式定义类，让解释器把调用分派给正确的方法，而不使用
if/elif/elif 块硬编码分派逻辑。

## 继承抽象基类的方法

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [ ]:
# 定义类的时候不会检查继承关系，但是当实例化的时候，会进行检查：
deck = FrenchDeck2()

导入时(加载并编译 frenchdeck2.py 模块时)，Python 不会检查抽象方
法的实现，在运行时实例化 FrenchDeck2 类时才会真正检查。因此，
如果没有正确实现某个抽象方法，Python 会抛出 TypeError 异常，并
把错误消息设为"Can't instantiate abstract class
FrenchDeck2 with abstract methods \_\_delitem\_\_, insert"。

正是
这个原因，即便 FrenchDeck2 类不需要 \_\_delitem\_\_ 和 insert
提供的行为，也要实现，因为 MutableSequence 抽象基类需要它 们。

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    # 实现抽象类的抽象方法
    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards[position].insert(position, value)

## 标准库中的抽象基类

从 Python 2.6 开始，标准库提供了抽象基类。大多数抽象基类在
collections.abc 模块中定义，不过其他地方也有。

如，numbers 和 io 包中有一些抽象基类。但是，collections.abc
中的抽象基类最常用。我们来看看这个模块中有哪些抽象基类。

# 自定义抽象类和抽象方法（自定义接口！）

In [85]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""


    @abc.abstractmethod
    def pick(self):
        """
            随机删除元素，然后将其返回。
            如果实例为空，这个方法应该抛出‘LookuppError’
        """

    def loaded(self):
        """
            如果至少有一个元素，返回‘True’，否则返回‘False’
        """
        return bool(self.inspect())

    def inspect(self):
        """
            返回一个有序元组，由当前元素构成。
        """
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [88]:
dir(Tombola)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'inspect',
 'load',
 'loaded',
 'pick']

In [91]:
Tombola._abc_impl

## 一个普通的继承，只实现抽象类中的抽象函数

In [9]:
# 一个子类： 使用一个更好的随机发生器

import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)


    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [10]:
f = BingoCage(['a','b','c','d'])

In [11]:
f.pick()

'a'

## 一个复杂的继承，不仅实现了抽象类中的抽象方法，还覆盖了抽象类中的非抽象方法，让代码效率提升

In [15]:
# 另一个子类：覆盖抽象类中的非抽象方法，提高效率

import random

class LotterBlower(Tombola):

    def __init__(self, iterable):
        self._ball = list(iterable) # 这里用到了4.2的方法，创建一个副本，避免使用传入的序列，产生混淆。（尤其是我们的类还要删除和增添该数组）

    def load(self, iterable):
        self._ball.extend(iterable)

    def pick(self):
        try:
            index = random.randrange(len(self._ball))
        except ValueError:
            raise LookupError("pick from empty LotterBlower!")
        return self._ball.pop(index)

    # 覆盖重构抽象类中已经实现的非抽象方法，来提高函数效率。
    def loaded(self):
        return bool(self._ball)

    # 覆盖重构抽象类中已经实现的非抽象方法，来提高函数效率。
    def inspect(self):
        return tuple(sorted(self._ball))

In [16]:
lb = LotterBlower('a b c d'.split())

In [17]:
lb.inspect()

('a', 'b', 'c', 'd')

In [18]:
lb.pick()

'c'

In [19]:
lb.loaded()

True

## 虚拟子类的使用方法

便不继 承，也有办法把一个类注册为抽象基类的虚拟子类。这样做时，我们保 证注册的类忠实地实现了抽象基类定义的接口，而 Python 会相信我们， 从而不做检查。如果我们说谎了，那么常规的运行时异常会把我们捕 获。

注册虚拟子类的方式是在抽象基类上调用 register 方法。这么做之 后，注册的类会变成抽象基类的虚拟子类，而且 issubclass 和
isinstance 等函数都能识别，但是注册的类不会从抽象基类中继承 任何方法或属性。


虚拟子类不会继承注册的抽象基类，而且任何时候都不会检
 查它是否符合抽象基类的接口，即便在实例化时也不会检查。为了
 避免运行时错误，虚拟子类要实现所需的全部方法。

In [70]:
# 使用注册的方法来实现虚拟子类有两种方法：一个是使用修饰器，一个是使用register函数

from random import randrange

# 使用第一种方法
@Tombola.register  # 使用基类的register修饰器来创建虚拟子类
class TombolaList(list):    # 实际继承了list类，亦可称为对list类的扩展

    # def __init__(self):   # 直接继承list，不需要重新实现初始化方式

    # 下面需要实现抽象基类的说有方法：
    def pick(self):         # 这里直接使用了list的__bool_方法
        if self:              
            index = randrange(len(self))
            return self.pop(index)
        else:
            raise LookupError("pick from empty TombolaList!")

    load = list.extend # 注意！这里直接将list的extent方法赋值给了虚拟子函数，因为他们的功能是一样的，省去了重新定义！！！
    # load = self.extend # 不能使用self.extend,会返回错误：name 'self' is not defined
                         # 而且在逻辑上也是不对的，毕竟self是该子函数，而不是父类。

    def loaded(self):
        return bool(list)

    def inspect(self):
        return tuple(sorted(self))


In [71]:
tbl = TombolaList('a b c d e'.split())

tbl.inspect()

('a', 'b', 'c', 'd', 'e')

In [72]:
tbl.load('x y z'.split())

In [73]:
tbl.inspect()

('a', 'b', 'c', 'd', 'e', 'x', 'y', 'z')

In [74]:
tbl.pick()

'x'

In [75]:
tbl.loaded()

True

In [76]:
# 另一种注册方式：
# 使用register函数

from random import randrange

class TombolaList2(list):    # 不再使用修饰器，而是使用函数，其他的都说是一样的

    # 下面需要实现抽象基类的说有方法：
    def pick(self):         # 这里直接使用了list的__bool_方法
        if self:              
            index = randrange(len(self))
            return self.pop(index)
        else:
            raise LookupError("pick from empty TombolaList!")

    load = list.extend 

    def loaded(self):
        return bool(list)

    def inspect(self):
        return tuple(sorted(self))
 
# 在全局进行注册： 
Tombola.register(TombolaList2)


__main__.TombolaList2

In [77]:
issubclass(TombolaList, Tombola)

True

In [78]:
issubclass(TombolaList2, Tombola)


True

In [79]:
issubclass(TombolaList2, list)

True

In [80]:
isinstance(tbl, Tombola)

True

In [81]:
isinstance(tbl, TombolaList)


True

In [82]:
isinstance(tbl, list)



True

In [83]:
isinstance(tbl, TombolaList2)

False

类的继承关系在一个特殊的类属性中指定—— __mro__，即方 法解析顺序(Method Resolution Order)。这个属性的作用很简单，按顺
序列出类及其超类，Python 会按照这个顺序搜索方法。17 查看 TomboList 类的 __mro__ 属性，你会发现它只列出了“真实的”超
类，即 list 和 object

In [84]:
TombolaList.__mro__

(__main__.TombolaList, list, object)

Tombolist.__mro__ 中没有 Tombola，因此 Tombolist 没有从 Tombola 中继承任何方法。
